# Lab 3: Creating Branches and Applying Cuts

## Table of Contents
1. **[Creating Branches](#creating-branches)**  
    1.1. [Converting Momentum Coordinate Systems](#conv-momentum)  
    1.2. [Calculating ($P_x, P_y, P_z$)](#calc-momentum)  
    1.3. [Exercise](#exercise-1.3)
2. **[Cuts on TTree Branches](#cuts)**  
    2.1. [Applying Cuts](#apply-cuts)  
    2.2. [Exercise](#exercise-2.2)

## 1. Creating Branches <a name="creating-branches" />
Often, you find you will want to create a new distribution from existing distributions in a ROOT file to calculate certain physical values that aren't already calculated. This can involve transforming the coordinate system of the momentum, calculating invariant masses of particles, or writing the classification of an event from the decision of a neural network.

### 1.1 Transforming Momentum Coordinate System <a name="trans-momentum" />
The detector does not store the momentum of each particle as $(P_x, P_y, P_z)$. Instead, momentum information is stored as $(P_t, \phi, \eta)$. These coordinate systems are related by the following equations:
$$P_x = P_t \times cos(\phi)$$
$$P_y = P_t \times sin(\phi)$$
$$P_z = P_t \times sinh(\eta)$$

In [1]:
TFile * root_file = TFile::Open("../../Datasets/TTbar/ee_ttbarsignalplustau.root");
TTree * tree = (TTree *) root_file->Get("ttBar_treeVariables_step8;4");
printf("TTree imported!");

TTree imported!

Now we have a `TTree` with all of the branches from the `TFile`'s `TTree`. We will use the `std::map` object for storing the calculated $P_x, P_y,$ and $P_z$ momentums. The `key` will be a `std::string` object representing the particle's component, e.g. `"top_px`, and the element will be a `std::vector<float>` as this will be the container used to store all of our particle's component data. A `std::vector` is being used as the size grows dynamically and is therefore a nice container for values. We need to grab the $P_t$, $\phi$, and $\eta$ for the $t, \bar{t}, b, \bar{b}, l$, and $\bar{l}$. Instead of typing each particle's name by hand, we will fill a `std::vector<string>` with these branch names that we need.

In [2]:
#include <vector>
#include <string>
#include <map>
using namespace std;

map<string, vector<float>> cartesianDistributions;
vector<string> particles{"top", "tbar", "l", "lbar", "b", "bbar"};

for(string particle : particles) {
    cartesianDistributions.insert(pair<string, vector<float>>(particle + "_px", *(new vector<float>{})));
    cartesianDistributions.insert(pair<string, vector<float>>(particle + "_py", *(new vector<float>{})));
    cartesianDistributions.insert(pair<string, vector<float>>(particle + "_pz", *(new vector<float>{})));
}

Now we need $(P_t, \phi, \eta)$ for each particle. Let's get these distributions from the `TTree`.

In [3]:
using namespace std;

vector<string> components{"_pt", "_phi", "_eta"};

map<string, vector<float>> distributions;

for(string particle : particles) {
    for(string component : components) {
        if((particle.compare("top") == 0 || particle.compare("tbar") == 0) && component.compare("_eta") == 0) {
            component = "_rapidity"; // top quark eta is labeled rapidity...means same thing
        }
        Float_t f = 0.0;
        string particleComponent = particle + component;
        tree->SetBranchAddress(particleComponent.c_str(), &f);
        Int_t nEntries = (Int_t) tree->GetEntries();
        vector<float> partiCompoDistr; // This will hold all of our data for all of our particles' component
        for(Int_t i = 0; i < nEntries; ++i) {
            tree->GetEntry(i);
            partiCompoDistr.push_back(f);
        }
        distributions.insert(pair<string, vector<float>>(particleComponent, partiCompoDistr));
    }
}


Now that we have all of the vector components loaded for all of our events, we can start performing our transformation of coordinate systems.

### 1.2 Calculating ($P_x, P_y, P_z$) <a name="calc-momentum" />
Now that we have the distributions, we can perform calculations. We will work with each particle individually and will be calculating the $P_x, P_y,$ and $P_z$. While we are performing these calculations we will be writing them to a `TTree`. Afterwards, we will save the `TTree` as a `TFile` and write that file to our hard drive.

_Note: This seems to not work within Jupyter, but does work outside of Jupyter. I am thinking it may be due to memory issues..._

In [4]:
#include <math.h>

TFile outputFile("ee_ttbarsignalplustau_cartesian.root", "RECREATE");
TTree cartesianTree("CartesianTree", "Tree that has Cartesian vector components for our particles");

/*
TODO: Make this work with a for loop like commented on below. For some reason this adds 6x the number of entries
It's probably not a coincidence we have 6 particles and its filling 6x number of entries. Not sure why though.
Brownie points to whoever figures it out...
*/
vector<float> top_pts = distributions["top_pt"];
vector<float> tbar_pts = distributions["tbar_pt"];
vector<float> top_phis = distributions["top_phi"];
vector<float> tbar_phis = distributions["tbar_phi"];
vector<float> top_etas = distributions["top_rapidity"];
vector<float> tbar_etas = distributions["tbar_rapidity"];

vector<float> l_pts = distributions["l_pt"];
vector<float> lbar_pts = distributions["lbar_pt"];
vector<float> l_phis = distributions["l_phi"];
vector<float> lbar_phis = distributions["lbar_phi"];
vector<float> l_etas = distributions["l_eta"];
vector<float> lbar_etas = distributions["lbar_eta"];

vector<float> b_pts = distributions["b_pt"];
vector<float> bbar_pts = distributions["bbar_pt"];
vector<float> b_phis = distributions["b_phi"];
vector<float> bbar_phis = distributions["bbar_phi"];
vector<float> b_etas = distributions["b_eta"];
vector<float> bbar_etas = distributions["bbar_eta"];

float top_px, top_py, top_pz = 0;
float tbar_px, tbar_py, tbar_pz = 0;
float l_px, l_py, l_pz = 0;
float lbar_px, lbar_py, lbar_pz = 0;
float b_px, b_py, b_pz = 0;
float bbar_px, bbar_py, bbar_pz = 0;

// Create the branches...
cartesianTree.Branch("top_px", &top_px, "top_px/F");
cartesianTree.Branch("top_py", &top_py, "top_py/F");
cartesianTree.Branch("top_pz", &top_pz, "top_pz/F");

cartesianTree.Branch("tbar_px", &tbar_px, "tbar_px/F");
cartesianTree.Branch("tbar_py", &tbar_py, "tbar_py/F");
cartesianTree.Branch("tbar_pz", &tbar_pz, "tbar_pz/F");

cartesianTree.Branch("l_px", &l_px, "l_px/F");
cartesianTree.Branch("l_py", &l_py, "l_py/F");
cartesianTree.Branch("l_pz", &l_pz, "l_pz/F");

cartesianTree.Branch("lbar_px", &lbar_px, "lbar_px/F");
cartesianTree.Branch("lbar_py", &lbar_py, "lbar_py/F");
cartesianTree.Branch("lbar_pz", &lbar_pz, "lbar_pz/F");

cartesianTree.Branch("b_px", &b_px, "b_px/F");
cartesianTree.Branch("b_py", &b_py, "b_py/F");
cartesianTree.Branch("b_pz", &b_pz, "b_pz/F");

cartesianTree.Branch("bbar_px", &bbar_px, "bbar_px/F");
cartesianTree.Branch("bbar_py", &bbar_py, "bbar_py/F");
cartesianTree.Branch("bbar_pz", &bbar_pz, "bbar_pz/F");

// There are the same number of entries in all of them, so we can just use top_pts to cycle through
for(int i = 0; i < top_pts.size(); i++) {
    top_px = top_pts[i] * cos(top_phis[i]);
    top_py = top_pts[i] * sin(top_phis[i]);
    top_pz = top_pts[i] * sinh(top_etas[i]);
    
    tbar_px = tbar_pts[i] * cos(tbar_phis[i]);
    tbar_py = tbar_pts[i] * sin(tbar_phis[i]);
    tbar_pz = tbar_pts[i] * sinh(tbar_etas[i]);

    l_px = l_pts[i] * cos(l_phis[i]);
    l_py = l_pts[i] * sin(l_phis[i]);
    l_pz = l_pts[i] * sinh(l_etas[i]);
    
    lbar_px = lbar_pts[i] * cos(lbar_phis[i]);
    lbar_py = lbar_pts[i] * sin(lbar_phis[i]);
    lbar_pz = lbar_pts[i] * sinh(lbar_etas[i]);
    
    b_px = b_pts[i] * cos(b_phis[i]);
    b_py = b_pts[i] * sin(b_phis[i]);
    b_pz = b_pts[i] * sinh(b_etas[i]);
    
    bbar_px = bbar_pts[i] * cos(bbar_phis[i]);
    bbar_py = bbar_pts[i] * sin(bbar_phis[i]);
    bbar_pz = bbar_pts[i] * sinh(bbar_etas[i]); 
    
    cartesianTree.Fill();
} 
                                       
cartesianTree.Write("decayTree", TObject::kOverwrite);

Now we can open a `TBrowser` through the interactive ROOT with our terminal and look at the results! You will see that the z momentum follows a different distribution than the x and y momentums. You will also see a dip in the momentum of the lepton around zero.

_This is how it should be written...much cleaner ...just adds 6x more entries..._

In [ ]:
#include <math.h>

TFile outputFile("ee_ttbarsignalplustau_cartesian.root", "RECREATE");
TTree cartesianTree("CartesianTree", "Tree that has Cartesian vector components for our particles");

for(string particle : particles) {
    // get our pt's, eta's, and phi's
    vector<float> pts = distributions[particle + "_pt"];
    vector<float> phis = distributions[particle + "_phi"];
    vector<float> etas;
    if(particle.compare("top") == 0 || particle.compare("tbar") == 0) {
        etas = distributions[particle + "_rapidity"];
    } else {
        etas = distributions[particle + "_eta"];
    }
    printf("The size of the %s pts is %i, phis is %i, and etas is %i. \n", particle.c_str(), pts.size(), phis.size(), etas.size());
    // Create some variables to represent the px, py, and pz
    float px, py, pz = 0;
    
    // Create the branches...
    cartesianTree.Branch((particle + "_px").c_str(), &px, "px/F");
    cartesianTree.Branch((particle + "_py").c_str(), &py, "py/F");
    cartesianTree.Branch((particle + "_pz").c_str(), &pz, "pz/F");

    // Now store them in our other vector<float> objects...
    for(int i = 0; i < pts.size(); ++i) {
        cartesianTree.GetEntry(i);
        px = pts[i] * cos(phis[i]);
        py = pts[i] * sin(phis[i]);
        pz = pts[i] * sinh(etas[i]);
        cartesianTree.Fill();
    }
}
// Write the TTree
cartesianTree.Write("decayTree", TObject::kOverwrite);

// Write ROOT file
outputFile.Write();
outputFile.Close();

### 1.3 Exercise <a name="exercise-1.3" />
With the ROOT file created above, calculate the magnitude of the momentum for each particle and create a new ROOT file with your new distributions.

In [ ]:
// Code here...?? (might be best to make a script)

## 2. Cuts on TTree Branches <a name="cuts" />

During an analysis, one of your objectives is to remove background from your data while retaining signal. Removing events from your data is called applying "cuts". In this lab, we will not discuss how to decide what is a good cut. We will simply go over how to apply them and create an output ROOT file.

### 2.1 Applying Cuts <a name="apply-cuts" />
In ROOT, cuts are given as strings. You define the branch you wish to cut on, give a boolean operator, and the value you wish to cut at. You can use ``&&`` to pass multiple cuts at once. An example of a cut string is ``"top_pt > 0 && tbar_pt > 0"``. You can apply no cut by defining ``selection = "1"``. Let's apply the first cut to our ROOT file.

In [1]:
// Define what cut to apply to the data
string selection = "top_pt >= 100 && tbar_pt >= 100";

// Retreieve ROOT file and TTree
TFile * sourceFile = TFile::Open("../../Datasets/TTbar/ee_ttbarsignalplustau.root");
TTree * sourceTree = (TTree *) sourceFile->Get("ttBar_treeVariables_step8");

// Create Destination ROOT file for TTree
TFile newFile("../../Datasets/TTbar/ee_ttbarsignalplustau_ttbar_pt_cut.root", "RECREATE");

// Cut and Copy Tree
TTree * cutTree = sourceTree->CopyTree(selection.c_str());

// Write cut Tree to a ROOT file
cutTree->Write("DecayTree", TObject::kWriteDelete);

// Write ROOT file
newFile.Write();
newFile.Close();

``TTree.CopyTree(selection)`` is a TTree method which returns a copy of a TTree with the cut defined in selection applied. It is important that you create the output root file, before you call ``TTree.CopyTree()``. When you call ``TTree.CopyTree()``, ROOT internally assigns the returned TTree to a ROOT file. If you call ``TTree.CopyTree()`` before you create the output ROOT file, the TTree will not be assigned to the output ROOT file and cannot be written.

After a cut you will have less events than you started with. Thus we define the efficiency of a cut as: $$efficiency = \frac{N_{after}}{N_{before}}$$
where $N$ is the number of events. During an analysis you will want to keep track of the effeciency of every cut you apply. You could go into TBrowser and look at the number of entries at every stage, but is easier to have your script give you the efficiency. ``TTree.GetEntries()`` will give you the total number of events in a tree. We can calculate the efficiency quickly in our script:

In [2]:
float nEntriesBefore = (float) sourceTree->GetEntries();
// We need to get the cutTree again because it is a virtual TTree
cutTree = sourceTree->CopyTree(selection.c_str());
float nEntriesAfter = (float) cutTree->GetEntries();
float efficiency = nEntriesAfter / nEntriesBefore;
printf("The efficiency of %s is: %f", selection.c_str(), efficiency);

The efficiency of top_pt >= 100 && tbar_pt >= 100 is: 0.581624

Usually this efficiency is in terms of your signal, as was stated earlier. So cuts will drastically reduce the efficiency of your background data, but should not effect the efficiency of your signal data.


### 2.2 Exercise <a name="exercise-2.2" />
Starting from the original ROOT file ``ee_ttbarsignalplustau.root``, apply the following cuts:
$$
\begin{align*}
t_{pt} &> 0\\
t_{pt} &< 800\\
\bar{t}_{pt} &> 0\\
\bar{t}_{pt} &< 800\\
\end{align*}
$$

Then find the efficiency of each cut at each stage of selection, and the total combined efficiency of all the cuts.  

Hint: ``TTree.GetEntries()`` can also take a cut as an argument and give you the number of events which pass the cut. This bypasses having 4 ROOT files with different cut levels.

In [ ]:
// Code goes here